<a href="https://colab.research.google.com/github/thejeswar2419/StudentGitLabRepo/blob/main/chapter_appendix-tools-for-deep-learning/jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from google.colab import files

print("Upload your YOLO dataset zip (with images/ and labels/ inside)...")
uploaded = files.upload()

# Take the first uploaded file
zip_filename = next(iter(uploaded))
print("Uploaded file:", zip_filename)

DATASET_ZIP = f"/content/{zip_filename}"
print("Dataset zip path in Colab:", DATASET_ZIP)


In [ ]:
import os

# Where to extract the ZIP inside Colab
EXTRACT_DIR = "/content/dataset_extracted"

# Inside the extracted folder:
# Change these if your zip has a structure like "valid/images", "valid/labels"
IMAGES_SUBDIR = "images"
LABELS_SUBDIR = "labels"

# Where to save annotated images in Google Drive
OUT_DIR = "/content/drive/MyDrive/deepui/Annotated_dataset/valid/docs"

# Optional: classes.txt inside the extracted root
CLASS_NAMES_FILE = os.path.join(EXTRACT_DIR, "classes.txt")

print("Will extract to:", EXTRACT_DIR)
print("Will save annotated images to:", OUT_DIR)

os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)


In [ ]:
import zipfile

# Extract the uploaded ZIP
print("Extracting zip...")
with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

# Build full paths
IMAGES_DIR = os.path.join(EXTRACT_DIR, IMAGES_SUBDIR)
LABELS_DIR = os.path.join(EXTRACT_DIR, LABELS_SUBDIR)

print("Images dir:", IMAGES_DIR)
print("Labels dir:", LABELS_DIR)

if not os.path.isdir(IMAGES_DIR):
    raise FileNotFoundError(f"Images directory not found: {IMAGES_DIR}")

if not os.path.isdir(LABELS_DIR):
    raise FileNotFoundError(f"Labels directory not found: {LABELS_DIR}")


In [ ]:
# Define your class names here (index = class id)
class_names = [
    "button",
    "image",# 0
    "text",    # 1
           # 4
    # add/remove as needed
]

print("Loaded class names:", class_names)
print("Number of classes:", len(class_names))


In [ ]:
import cv2

# ---------- CONFIG ----------
TARGET_SIZE = (1080, 2400)  # (width, height)

# Color palette for classes
PALETTE = [
    (31,119,180),(255,127,14),(44,160,44),(214,39,40),(148,103,189),
    (140,86,75),(227,119,194),(127,127,127),(188,189,34),(23,190,207)
]

def color_for_class(cid):
    return PALETTE[cid % len(PALETTE)]

def yolo_to_bbox(xc, yc, w, h, img_w, img_h):
    # xc,yc,w,h are normalized (0..1)
    cx = xc * img_w
    cy = yc * img_h
    bw = w * img_w
    bh = h * img_h
    x1 = int(round(cx - bw/2))
    y1 = int(round(cy - bh/2))
    x2 = int(round(cx + bw/2))
    y2 = int(round(cy + bh/2))
    # clamp
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(img_w - 1, x2)
    y2 = min(img_h - 1, y2)
    return x1, y1, x2, y2

# Collect image files
img_exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
files = [f for f in os.listdir(IMAGES_DIR) if os.path.splitext(f.lower())[1] in img_exts]
files.sort()

if not files:
    print("No image files found in", IMAGES_DIR)
else:
    print(f"Found {len(files)} images.")

target_w, target_h = TARGET_SIZE

for fname in files:
    img_path = os.path.join(IMAGES_DIR, fname)
    base = os.path.splitext(fname)[0]
    label_path = os.path.join(LABELS_DIR, base + ".txt")

    img = cv2.imread(img_path)
    if img is None:
        print("Warning: cannot read image:", img_path)
        continue

    orig_h, orig_w = img.shape[:2]

    # Resize image
    resized = cv2.resize(img, (target_w, target_h), interpolation=cv2.INTER_AREA)

    # If no label file, just save resized image
    if not os.path.isfile(label_path):
        out_path = os.path.join(OUT_DIR, fname)
        cv2.imwrite(out_path, resized)
        print("No label for", fname, "-> saved resized only.")
        continue

    # Read labels
    with open(label_path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f if ln.strip()]

    annotated = resized.copy()

    # Detect if labels are absolute pixel coords
    is_absolute = False
    for ln in lines:
        parts = ln.split()
        if len(parts) >= 5:
            try:
                nums = list(map(float, parts[1:5]))
                if max(nums) > 1.0:
                    is_absolute = True
                    break
            except:
                pass

    for ln in lines:
        parts = ln.split()
        if len(parts) < 5:
            print(f"Malformed label in {label_path}: {ln}")
            continue

        try:
            cid = int(parts[0])
            v1, v2, v3, v4 = map(float, parts[1:5])
        except ValueError:
            print(f"Value error in {label_path}: {ln}")
            continue

        if is_absolute:
            # Convert from absolute pixels (original) to normalized
            xc_norm = v1 / orig_w
            yc_norm = v2 / orig_h
            w_norm  = v3 / orig_w
            h_norm  = v4 / orig_h
        else:
            xc_norm, yc_norm, w_norm, h_norm = v1, v2, v3, v4

        x1, y1, x2, y2 = yolo_to_bbox(xc_norm, yc_norm, w_norm, h_norm, target_w, target_h)

        color = color_for_class(cid)
        thickness = max(1, int(round(min(target_w, target_h) / 200)))
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, thickness)

        # Prepare label text
        label_text = str(cid)
        if 0 <= cid < len(class_names):
            label_text = f"{cid}: {class_names[cid]}"

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.45
        (tw, th), baseline = cv2.getTextSize(label_text, font, font_scale, 1)
        tx1 = x1
        ty1 = max(0, y1 - th - baseline - 3)
        tx2 = tx1 + tw + 6
        ty2 = ty1 + th + baseline + 3

        cv2.rectangle(annotated, (tx1, ty1), (tx2, ty2), color, -1)
        cv2.putText(annotated, label_text, (tx1 + 3, ty2 - baseline - 2),
                    font, font_scale, (255, 255, 255), 1, cv2.LINE_AA)

    out_path = os.path.join(OUT_DIR, fname)
    cv2.imwrite(out_path, annotated)
    print("Saved annotated:", out_path)

print("✅ All done! Annotated images are in:", OUT_DIR)
